In [ ]:
import os
import pickle
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
%matplotlib inline

import keras
from keras.models import Sequential
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Flatten, Dense, Activation, Reshape

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# pre-trained weights require this ordering
keras.backend.set_image_dim_ordering("th")

In [ ]:
def get_model():
    model = Sequential()

    # layer 1
    model.add(Convolution2D(16, 3, 3, input_shape=(3, 448, 448), border_mode="same", subsample=(1, 1)))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    # Layer 2
    model.add(Convolution2D(32, 3, 3, border_mode="same"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), border_mode="valid"))

    # Layer 3
    model.add(Convolution2D(64, 3, 3, border_mode="same"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), border_mode="valid"))

    # Layer 4
    model.add(Convolution2D(128, 3, 3, border_mode="same"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), border_mode="valid"))

    # Layer 5
    model.add(Convolution2D(256, 3, 3, border_mode="same"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), border_mode="valid"))

    # Layer 6
    model.add(Convolution2D(512, 3, 3, border_mode="same"))
    model.add(LeakyReLU(alpha=0.1))
    model.add(MaxPooling2D(pool_size=(2, 2), border_mode="valid"))

    # Layer 7
    model.add(Convolution2D(1024, 3, 3, border_mode="same"))
    model.add(LeakyReLU(alpha=0.1))

    # Layer 8
    model.add(Convolution2D(1024, 3, 3, border_mode="same"))
    model.add(LeakyReLU(alpha=0.1))

    # Layer 9
    model.add(Convolution2D(1024, 3, 3, border_mode="same"))
    model.add(LeakyReLU(alpha=0.1))

    model.add(Flatten())

    # Layer 10
    model.add(Dense(256))

    # Layer 11
    model.add(Dense(4096))
    model.add(LeakyReLU(alpha=0.1))

    # Layer 12
    model.add(Dense(1470))

    return model


model = get_model()
model.summary()

In [ ]:
def resize(image):
    PIL_image = Image.fromarray(image)
    resized = PIL_image.resize((448, 448), Image.LANCZOS)
    return np.array(resized)


def normalize(image):
    normalized = 2. * image / 255. - 1.
    return normalized


def preprocess(image):
    resized = resize(image)
    normalized = normalize(resized)
    # model works on (channel, height, width) ordering of dimensions
    transposed = np.transpose(normalized, (2, 0, 1))
    return transposed


class Box:
    def __init__(self):
        self.x, self.y = float(), float()
        self.w, self.h = float(), float()
        self.c = float()
        self.prob = float()

        
def overlap(x1, w1, x2, w2):
    l1 = x1 - w1 / 2.
    l2 = x2 - w2 / 2.
    left = max(l1, l2)
    r1 = x1 + w1 / 2.
    r2 = x2 + w2 / 2.
    right = min(r1, r2)
    return right - left


# return intersection of box a and box b
def box_intersection(a, b):
    w = overlap(a.x, a.w, b.x, b.w)
    h = overlap(a.y, a.h, b.y, b.h)
    if w < 0 or h < 0:
        return 0
    area = w * h
    return area


# return area under union of box a and box b
def box_union(a, b):
    i = box_intersection(a, b)
    u = a.w * a.h + b.w * b.h - i
    return u


# return intersection over union of box a and box b
def box_iou(a, b):
    return box_intersection(a, b) / box_union(a, b)


def yolo_output_to_car_boxes(yolo_output, threshold=0.2, sqrt=1.8, C=20, B=2, S=7):
    # position for class car in the VOC dataset classes
    car_class_number = 6

    boxes = []
    SS = S * S  # number of grid cells
    prob_size = SS * C  # class probabilities
    conf_size = SS * B  # confidences for each grid cell

    probabilities = yolo_output[0:prob_size]
    confidence_scores = yolo_output[prob_size: (prob_size + conf_size)]
    cords = yolo_output[(prob_size + conf_size):]

    # reshape the arrays so that its easier to loop over them
    probabilities = probabilities.reshape((SS, C))
    confs = confidence_scores.reshape((SS, B))
    cords = cords.reshape((SS, B, 4))

    for grid in range(SS):
        for b in range(B):
            bx = Box()

            bx.c = confs[grid, b]

            # bounding box xand y coordinates are offsets of a particular grid cell location,
            # so they are also bounded between 0 and 1.
            # convert them absolute locations relative to the image size
            bx.x = (cords[grid, b, 0] + grid % S) / S
            bx.y = (cords[grid, b, 1] + grid // S) / S


            bx.w = cords[grid, b, 2] ** sqrt
            bx.h = cords[grid, b, 3] ** sqrt

            # multiply confidence scores with class probabilities to get class sepcific confidence scores
            p = probabilities[grid, :] * bx.c

            # Check if the confidence score for class car is greater than the threshold
            if p[car_class_number] >= threshold:
                bx.prob = p[car_class_number]
                boxes.append(bx)

    # combine boxes that overlap
    boxes.sort(key=lambda b: b.prob, reverse=True)
    for i in range(len(boxes)):
        boxi = boxes[i]
        if boxi.prob == 0:
            continue

        for j in range(i + 1, len(boxes)):
            boxj = boxes[j]

            # if boxes have more than 40% overlap then retain the box with the highest confidence score
            if box_iou(boxi, boxj) >= 0.4:
                boxes[j].prob = 0

    boxes = [b for b in boxes if b.prob > 0]
    return boxes


def draw_boxes(boxes, im, crop_dim):
    PIL_image = Image.fromarray(im.copy())
    draw = ImageDraw.Draw(PIL_image)
    [xmin, xmax] = crop_dim[0]
    [ymin, ymax] = crop_dim[1]
    
    height, width, _ = im.shape
    for b in boxes:
        w = xmax - xmin
        h = ymax - ymin

        left = int((b.x - b.w / 2.) * w) + xmin
        right = int((b.x + b.w / 2.) * w) + xmin
        top = int((b.y - b.h / 2.) * h) + ymin
        bot = int((b.y + b.h / 2.) * h) + ymin

        if left < 0:
            left = 0
        if right > width - 1:
            right = width - 1
        if top < 0:
            top = 0
        if bot > height - 1: 
            bot = height - 1
        
        thickness = 3
        draw.line([(left, bot), (left, top)], fill=(255, 0, 0), width=thickness)
        draw.line([(left, top), (right, top)], fill=(255, 0, 0), width=thickness)
        draw.line([(right, top), (right, bot)], fill=(255, 0, 0), width=thickness)
        draw.line([(right, bot), (left, bot)], fill=(255, 0, 0), width=thickness)

    del draw
    return np.array(PIL_image)

In [ ]:
def load_weights(model, yolo_weight_file):
    data = np.fromfile(yolo_weight_file, np.float32)
    data = data[4:]

    index = 0
    for layer in model.layers:
        shape = [w.shape for w in layer.get_weights()]
        if shape != []:
            kshape, bshape = shape
            bia = data[index:index + np.prod(bshape)].reshape(bshape)
            index += np.prod(bshape)
            ker = data[index:index + np.prod(kshape)].reshape(kshape)
            index += np.prod(kshape)
            layer.set_weights([ker, bia])
            

model = get_model()
load_weights(model, "../weights/yolo-tiny.weights")

In [ ]:
image = Image.open("../dataset/videos_drop_loss/1_30pct_drop/image_010.png")
image = image.convert("RGB")
image = np.array(image)
preprocessed = preprocess(image)
batch = np.expand_dims(preprocessed, axis=0)
batch_output = model.predict(batch)
boxes = yolo_output_to_car_boxes(batch_output[0], threshold=0.20)
final = draw_boxes(boxes, image, ((0, image.shape[1]),(0, image.shape[0])))

plt.imshow(final)
plt.axis("off")
plt.show()

## Drop loss evaluation

In [ ]:
annot_dict = pickle.load(open("../dataset/annot_dict.p", "rb"))
dir_prefix = "../dataset/videos_drop_loss"
drop_rates = ["10pct", "20pct", "30pct"]
video_names = ["1", "2", "3", "4", "5", "6"]

annot_dir_names = ["../dataset/1_annot", "../dataset/2_annot", "../dataset/3_annot",
                   "../dataset/4_annot", "../dataset/5_annot", "../dataset/6_annot"]
col_names = ["Video name", "Image no. (p=0.1)", "IoU (p=0.1)", "Conf. (p=0.1)", "Prob. (p=0.1)", "Image no. (p=0.2)",
             "IoU (p=0.2)", "Conf. (p=0.2)", "Prob. (p=0.2)", "Image no. (p=0.3)", "IoU (p=0.3)", "Conf. (p=0.3)", 
             "Prob. (p=0.3)"]
df = pd.DataFrame(index=range(len(video_names)), columns=col_names)

In [ ]:
kept_frames_df = pd.read_csv(os.path.join(dir_prefix, "frames_kept.csv"))

In [ ]:
results_dict = {}

for k in range(0, len(video_names)):
    video_name = video_names[k]
    annot_dir = annot_dir_names[k]

    for drop_rate in drop_rates:
        dir_name = os.path.join(dir_prefix, video_name + "_" + drop_rate + "_drop")
        
        num_images = 0
        for f in os.listdir(dir_name):
            if f.endswith(".png") and not f.startswith("."):
                num_images += 1

        print "Starting " + dir_name
        print
        
        mean_iou_all = []
        c_all = []
        prob_all = []
        results_dict[dir_name] = {}
        
        kept_frames = kept_frames_df[dir_name.split("/")[-1]].dropna().tolist()

        for i in range(1, num_images + 1):
            # make predictions
            image_name = "image_" + format(i, "03d") + ".png"
            image_file = os.path.join(dir_name, image_name)
            image = Image.open(image_file)
            image = image.convert("RGB")
            image = np.array(image)
            preprocessed = preprocess(image)
            batch = np.expand_dims(preprocessed, axis=0)
            batch_output = model.predict(batch)
            boxes = yolo_output_to_car_boxes(batch_output[0], threshold=0.20)

            # get annotations and compare
            annot_image_name = "image_" + format(int(kept_frames[i - 1]) + 1, "03d") + ".png"
            annot_boxes = annot_dict[annot_dir][annot_image_name]
            num_annot_boxes = len(annot_boxes)
            total_iou = 0.0
            for annot_box in annot_boxes:
                best_iou = 0.0
                for b in boxes:
                    curr_iou = box_iou(annot_box, b)
                    if curr_iou > best_iou:
                        best_iou = curr_iou
                total_iou += best_iou
            mean_iou = total_iou / num_annot_boxes
            mean_iou_all.append(mean_iou)

            confidences = []
            probabilities = []
            for b in boxes:
                confidences.append(b.c)
                probabilities.append(b.prob)

            if len(confidences) == 0:
                c_mean = 0.0
            else:
                c_mean = np.mean(confidences)
            if len(probabilities) == 0:
                prob_mean = 0.0
            else:
                prob_mean = np.mean(probabilities)

            c_all.append(c_mean)
            prob_all.append(prob_mean)

            if i % 10 == 0:
                print("Mean IoU: " + str(mean_iou))
                print("Mean confidence: " + str(c_mean))
                print("Mean probability: " + str(prob_mean))
                print "Done with image " + str(i) + " of " + str(num_images)
                print

        results_dict[dir_name]["iou"] = mean_iou_all
        results_dict[dir_name]["c"] = c_all
        results_dict[dir_name]["prob"] = prob_all

In [ ]:
for i, video_name in enumerate(video_names):
    df.iloc[i, 0] = video_name
    
    dir_name = os.path.join(dir_prefix, video_name + "_10pct_drop")
    df.iloc[i, 1] = len(results_dict[dir_name]["iou"])
    df.iloc[i, 2] = np.mean(results_dict[dir_name]["iou"])
    df.iloc[i, 3] = np.mean(results_dict[dir_name]["c"])
    df.iloc[i, 4] = np.mean(results_dict[dir_name]["prob"])
    
    dir_name = os.path.join(dir_prefix, video_name + "_20pct_drop")
    df.iloc[i, 5] = len(results_dict[dir_name]["iou"])
    df.iloc[i, 6] = np.mean(results_dict[dir_name]["iou"])
    df.iloc[i, 7] = np.mean(results_dict[dir_name]["c"])
    df.iloc[i, 8] = np.mean(results_dict[dir_name]["prob"])
    
    dir_name = os.path.join(dir_prefix, video_name + "_30pct_drop")
    df.iloc[i, 9] = len(results_dict[dir_name]["iou"])
    df.iloc[i, 10] = np.mean(results_dict[dir_name]["iou"])
    df.iloc[i, 11] = np.mean(results_dict[dir_name]["c"])
    df.iloc[i, 12] = np.mean(results_dict[dir_name]["prob"])

In [ ]:
df.to_csv("../results/drop_loss_df.csv", sep=",", index=False)

In [ ]:
pickle.dump(results_dict, open("../results/drop_loss_results.p", "wb"))